In [1]:
import numpy as np
import math
import scipy.fft as spfft

### Implementações que fiz durantes os estudos

podem me ajudar durante a tarefa

In [5]:
def dct1(x):
    N = len(x)
    m = np.arange(0,N,1)
    X = np.zeros(N)
    
    for k in range(N):
        X[k] = np.sqrt((1+(k>0))/N) * np.sum(x*np.cos((math.pi*k*(m+0.5))/N))
        
    return X

def idct1(C):
    N = len(C)
    x = np.zeros(N)
    k = np.arange(1,N,1)
    for m in range(N):
#         x[m] = np.sqrt(1/N)*C[0] + np.sqrt(2/N) * np.sum( np.multiply( C[1:] , np.cos( (math.pi*(m+0.5)*k)/N ) ))
        x[m] = np.sqrt(1/N)*C[0] + np.sqrt(2/N) * np.sum( C[1:] * np.cos( (math.pi*(m+0.5)*k)/N ) )

    return x

def build_DCT_base(N=4):
    res = np.zeros((N,N))
    k = np.arange(1, N, 1).reshape((N-1,1))
    m = np.arange(0, N, 1).reshape((1,N))
    fatores = k * m
    res[0:] = np.sqrt(1/N)
    res[1:] = np.sqrt(2/N) * np.cos(math.pi*k*(m+0.5)/N )
    return res

def dct2(x):
    N = len(x)
    base = build_DCT_base(N)
    res = np.matmul(base, x)
    return res

def idct2(C):
    N = len(C)
    base_t = build_DCT_base(N).T
    res = np.matmul(base_t, C)
    return res

def test_dct(dut, N=40):
    x = np.random.randn(N)
    
    golden = spfft.dct(x, norm='ortho')
    result = dut(x)
    
    assert np.allclose(golden, result) == True , f'Teste falhou! \nGOLDEN:\t{golden} \nDUT:\t{result}'
    print('Teste passou! ')
    
def test_idct(dut, N=4):
    C = np.random.randn(N)
    golden = spfft.idct(C, norm='ortho')
    result = dut(C)
    
    assert np.allclose(golden, result) == True, 'Teste falhou!'
    print('Teste passou!')    
    
test_dct(dct1, 100)
test_idct(idct1, 100)
test_dct(dct2, 100)
test_idct(idct2, 100)

Teste passou! 
Teste passou!
Teste passou! 
Teste passou!


In [9]:
def dct2d_1(A):
    m, n = A.shape
    
    base_m = build_DCT_base(m)
    base_n = build_DCT_base(n)
    
    tmp1 = np.matmul(base_m, A)
    
    return np.matmul(tmp1, base_n.T)
    
def idct2d_1(Ahat):
    m, n = Ahat.shape
    
    base_m = build_DCT_base(m)
    base_n = build_DCT_base(n)
    
    tmp1 = np.matmul(base_m.T, Ahat)
    
    return np.matmul(tmp1, base_n)
    

def test_dct2d(dut, M=100, N=40):
    A = np.random.randn(M*N).reshape((M,N))
    
    golden = spfft.dctn(A, norm='ortho')
    result = dut(A)
    
    assert np.allclose(golden, result) == True , 'Teste falhou!'
    print('Teste passou! ')

def test_idct2d(dut, M=100, N=40):
    Ahat = np.random.randn(M*N).reshape((M,N))
    
    golden = spfft.dctn(A, norm='ortho')
    result = dut(Ahat)
    
    assert np.allclose(golden, result) == True , 'Teste falhou!'
    print('Teste passou! ')
    
def dct2d_2(A):
    m, n = A.shape
    
    tmp1 = np.zeros((m,n))
    tmp2 = np.zeros((n,m))
    for row in range(m):
        #tmp1[row] = spfft.dct(A[row], norm='ortho')
        tmp1[row] = dct1(A[row])
    for col in range(n):
        tmp2[col] = dct1(tmp1.T[col])
    Ahat = tmp2.T
    return Ahat

def dct2d_3(A):
    M, N = A.shape
    
    A_hat = np.zeros((M,N))
    
    for k in range(M):
        for l in range(N):
            for r in range(M):
                for s in range(N):
                    norm_k = np.sqrt((1+(k>0))/M)
                    norm_l = np.sqrt((1+(l>0))/N)
                    A_hat[k][l] += norm_k * norm_l * A[r][s] * np.cos((math.pi*k*(r+0.5))/M) * np.cos((math.pi*l*(s+0.5))/N)
    return A_hat


def dct2d_4(m):
    D1 = spfft.dct(m.T, norm='ortho')
    D2 = spfft.dct(D1.T, norm='ortho')
    return D2

test_dct2d(dct2d_1,800,1234)
test_dct2d(dct2d_2,80,123)
test_dct2d(dct2d_3,20,12)
test_dct2d(dct2d_4,800,1234)


Teste passou! 
Teste passou! 
Teste passou! 
Teste passou! 
